# 08 — Feed Forward Network

A simple Feed Forward Neural Network (FFN) using **Word2Vec document vectors** as input.

**Architecture**: `100 → 64 → 32 → 1` (ReLU + Dropout + Sigmoid)

Trained on both **Standard** and **Irony** pipelines.

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, classification_report
import os

In [2]:
%load_ext watermark
%watermark -v -n -m -p numpy,torch,sklearn

Python implementation: CPython
Python version       : 3.12.12
IPython version      : 9.10.0

numpy  : 1.26.4
torch  : 2.2.2
sklearn: 1.8.0

Compiler    : Clang 17.0.0 (clang-1700.6.3.2)
OS          : Darwin
Release     : 25.2.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit



## 1. Model Definition

In [3]:
class FFN(nn.Module):
    def __init__(self, input_dim=100):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.network(x)

## 2. Training Function

In [4]:
def train_ffn(variation_name, w2v_dir, output_dir, epochs=20, lr=1e-3, batch_size=32):
    print(f"\n{'='*20} FFN: {variation_name} {'='*20}")
    
    # Load pre-computed Word2Vec document vectors
    X_train = np.load(f'{w2v_dir}/doc_vectors_train.npy')
    X_test  = np.load(f'{w2v_dir}/doc_vectors_test.npy')
    y_train = np.load(f'{w2v_dir}/labels_train.npy').astype(np.float32)
    y_test  = np.load(f'{w2v_dir}/labels_test.npy').astype(np.float32)
    
    print(f"Train: {X_train.shape}, Test: {X_test.shape}")
    
    # PyTorch datasets
    train_ds = TensorDataset(
        torch.FloatTensor(X_train),
        torch.FloatTensor(y_train)
    )
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    
    # Model
    model = FFN(input_dim=X_train.shape[1])
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Train
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            output = model(batch_x).squeeze()
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        if (epoch + 1) % 5 == 0:
            print(f"  Epoch {epoch+1}/{epochs} — Loss: {total_loss/len(train_loader):.4f}")
    
    # Evaluate
    model.eval()
    with torch.no_grad():
        X_test_t = torch.FloatTensor(X_test)
        preds = model(X_test_t).squeeze()
        y_pred = (preds >= 0.5).int().numpy()
    
    acc = accuracy_score(y_test, y_pred)
    print(f"\nFFN ({variation_name}) Accuracy: {acc:.4f}")
    print(classification_report(y_test.astype(int), y_pred))
    
    # Save
    os.makedirs(output_dir, exist_ok=True)
    torch.save(model.state_dict(), f'{output_dir}/model.pt')
    print(f"Model saved to {output_dir}/model.pt")
    
    return acc

## 3. Run Both Pipelines

In [5]:
torch.manual_seed(42)
np.random.seed(42)

acc_standard = train_ffn("Standard", "../models/word2vec/standard", "../models/ffn/standard")
acc_irony    = train_ffn("Irony",    "../models/word2vec/irony",    "../models/ffn/irony")


==================== FFN: Standard ====================
Train: (2100, 100), Test: (450, 100)


  Epoch 5/20 — Loss: 0.5221


  Epoch 10/20 — Loss: 0.4884


  Epoch 15/20 — Loss: 0.4698


  Epoch 20/20 — Loss: 0.4458

FFN (Standard) Accuracy: 0.7600
              precision    recall  f1-score   support

           0       0.78      0.73      0.75       225
           1       0.74      0.79      0.77       225

    accuracy                           0.76       450
   macro avg       0.76      0.76      0.76       450
weighted avg       0.76      0.76      0.76       450

Model saved to ../models/ffn/standard/model.pt

==================== FFN: Irony ====================
Train: (2100, 100), Test: (450, 100)


  Epoch 5/20 — Loss: 0.5259


  Epoch 10/20 — Loss: 0.4927


  Epoch 15/20 — Loss: 0.4726


  Epoch 20/20 — Loss: 0.4492

FFN (Irony) Accuracy: 0.7778
              precision    recall  f1-score   support

           0       0.78      0.77      0.78       225
           1       0.78      0.78      0.78       225

    accuracy                           0.78       450
   macro avg       0.78      0.78      0.78       450
weighted avg       0.78      0.78      0.78       450

Model saved to ../models/ffn/irony/model.pt


## 4. Comparison

In [6]:
print("\n=== Final Comparison ===")
print(f"Standard: {acc_standard:.4f}")
print(f"Irony:    {acc_irony:.4f}")
diff = acc_irony - acc_standard
print(f"Impact of Irony features: {diff:+.4f}")


=== Final Comparison ===
Standard: 0.7600
Irony:    0.7778
Impact of Irony features: +0.0178
